<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Setup

<div style="background-color:teal; color:white; padding:5px; font-size:20px">
Installs

In [1]:
!pip install hopsworks

  Using cached hopsworks-3.4.4-py3-none-any.whl
  Using cached hsfs-3.4.7-py3-none-any.whl
  Using cached hsml-3.4.6-py3-none-any.whl
  Using cached pyhumps-1.6.1-py3-none-any.whl (5.0 kB)
  Using cached furl-2.1.3-py2.py3-none-any.whl (20 kB)
  Using cached pyjks-20.0.0-py2.py3-none-any.whl (45 kB)
  Using cached avro-1.11.0-py2.py3-none-any.whl
  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached great_expectations-0.14.13-py3-none-any.whl (5.0 MB)
  Using cached MarkupSafe-2.0.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (30 kB)
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
  Using cached altair-4.2.2-py3-none-any.whl (813 kB)
  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached ruamel.yaml-0.17.17-py3-none-any.whl (109 kB)
  Using cached PyHopsHive-0.6.4.1.dev0-py3-none-any.whl
  Using cached confluent_kafka-2.3

In [2]:
!pip install tensorflow

  Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 3.0.1 requires Jinja2>=3.1.2, but you have jinja2 3.0.3 which is incompatible.
hsfs 3.4.7 requires markupsafe<2.1.0, but you have markupsafe 2.1.5 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.


In [3]:
!pip install tensorboard

In [4]:
!pip install tensorflow_recommenders

  Using cached tensorflow_recommenders-0.7.3-py3-none-any.whl (96 kB)


In [5]:
!pip install tensorflow_addons

  Using cached tensorflow_addons-0.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
Using cached tensorflow_addons-0.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (611 kB)


<div style="background-color:teal; color:white; padding:5px; font-size:20px">
Imports

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import StringLookup, Normalization
import datetime
import warnings
warnings.filterwarnings('ignore')

2024-02-16 19:07:18.738608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Hopsworks Feature Store Connection

In [4]:
import boto3

# Initialize a client for Systems Manager
ssm = boto3.client('ssm', region_name='us-east-1')

parameter_name = 'hopsworks-api-key'

# Fetch the parameter
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)

# Extract the parameter value (API key in this case)
hopsworks_api_key = response['Parameter']['Value']

In [7]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/475285
Connected. Call `.close()` to terminate connection gracefully.


<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Training Dataset Creation

<div style="background-color:teal; color:white; padding:5px; font-size:20px">
Create Dataset From Feature Store (Hopsworks)

In [9]:
# If View Already Created:
feature_view = fs.get_feature_view(name='retrieval', version=1)

In [10]:
# Fetch Datasets From View Using Hopsworks Train-test Split Function
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)

Finished: Reading data from Hopsworks, using ArrowFlight (115.15s) 


In [11]:
train_df["article_id"] = train_df["article_id"].astype(str)  # to be removed
val_df["article_id"] = val_df["article_id"].astype(str)  # to be removed

In [12]:
query_features = ["customer_id", "age", "month_sin", "month_cos"]
candidate_features = ["article_id", "garment_group_name", "index_group_name"]

def df_to_ds(df):
    # Convert df to tensoflow dataset
    return tf.data.Dataset.from_tensor_slices({col : df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(val_df).batch(BATCH_SIZE).cache()

Creating unique-element lists for categorical embeddings

In [13]:
user_id_list = train_df["customer_id"].unique().tolist()
item_id_list = train_df["article_id"].unique().tolist()

garment_group_list = train_df["garment_group_name"].unique().tolist()
index_group_list = train_df["index_group_name"].unique().tolist()

print(f"Number of transactions: {len(train_df):,}")
print(f"Number of users: {len(user_id_list):,}")
print(f"Number of items: {len(item_id_list):,}")
print(garment_group_list)

Number of transactions: 433,927
Number of users: 131,880
Number of items: 24,621
['Dressed', 'Jersey Basic', 'Blouses', 'Knitwear', 'Jersey Fancy', 'Trousers Denim', 'Swimwear', 'Trousers', 'Socks and Tights', 'Shoes', 'Skirts', 'Dresses Ladies', 'Accessories', 'Outdoor', 'Under-, Nightwear', 'Special Offers', 'Shirts', 'Dresses/Skirts girls', 'Shorts', 'Unknown', 'Woven/Jersey/Knitted mix Baby']


<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Retrieval Model

FactorizedTopK:
- Positive samples are only ones associated with the INDIVIDUAL query record. It does not reward for retrieving an item associated with the user on a separate record.

In [14]:
EMB_DIM = 16

<div style="background-color:teal; color:white; padding:5px; font-size:20px">
Query Tower Definition

In [15]:
class QueryTower(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        # user_embedding: Integer-Encoding
        self.user_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=user_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # We add an additional embedding to account for unknown tokens.
                len(user_id_list) + 1,
                EMB_DIM
            )
        ])

        # age: Normalization
        self.normalized_age = Normalization(axis=None)
        
        # Embedding Layer Definition
        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(EMB_DIM, activation="relu"),
            tf.keras.layers.Dense(EMB_DIM)
        ])

    def call(self, inputs):
        # Concat respectively transformed inputs
        concatenated_inputs = tf.concat([
            self.user_embedding(inputs["customer_id"]),
            tf.reshape(self.normalized_age(inputs["age"]), (-1,1)),
            tf.reshape(inputs["month_sin"], (-1,1)),
            tf.reshape(inputs["month_cos"], (-1,1))
        ], axis=1)

        # Send through embedding layer
        outputs = self.fnn(concatenated_inputs)

        return outputs

In [16]:
# Instantiate Model
query_model = QueryTower()

# Computing dataset-wide stats for normalization
query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Model Inputs:
query_df = train_df[query_features] # Subset to query feats
query_ds = df_to_ds(query_df).batch(1) # convert df to tf dataset

# load inputs into model
query_model(next(iter(query_ds))) 

2024-02-16 19:27:09.256283: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [433927]
	 [[{{node Placeholder/_1}}]]
2024-02-16 19:27:09.257085: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype double and shape [433927]
	 [[{{node Placeholder/_8}}]]
2024-02-16 19:27:19.896005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[-0.11535659, -0.38656187, -0.02634433, -0.20807752, -0.4647346 ,
        -0.22234225,  0.09918011,  0.07456864, -0.24407706, -0.13969278,
         0.07337382,  0.0697898 ,  0.17010647, -0.16232108,  0.12789005,
        -0.01841322]], dtype=float32)>

<div style="background-color:teal; color:white; padding:5px; font-size:20px">
Item Tower Definition

In [17]:
class ItemTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.item_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=item_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # We add an additional embedding to account for unknown tokens.
                len(item_id_list) + 1,
                EMB_DIM
            )
        ])

        self.garment_group_tokenizer = StringLookup(vocabulary=garment_group_list, mask_token=None)
        self.index_group_tokenizer = StringLookup(vocabulary=index_group_list, mask_token=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(EMB_DIM, activation="relu"),
            tf.keras.layers.Dense(EMB_DIM)
        ])

    def call(self, inputs):
        garment_group_embedding = tf.one_hot(
            self.garment_group_tokenizer(inputs["garment_group_name"]),
            len(garment_group_list)
        )

        index_group_embedding = tf.one_hot(
            self.index_group_tokenizer(inputs["index_group_name"]),
            len(index_group_list)
        )

        concatenated_inputs = tf.concat([
            self.item_embedding(inputs["article_id"]),
            garment_group_embedding,
            index_group_embedding
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs

In [18]:
# Instantiate Model
item_model = ItemTower()

# Model Inputs:
item_df = train_df[candidate_features] # Subset to item feats
item_df.drop_duplicates(subset="article_id", inplace=True) # Drop dup col
item_ds = df_to_ds(item_df) # Convert to tf dataset

<div style="background-color:teal; color:white; padding:5px; font-size:20px">
Two Tower Model Definition

In [19]:
import tensorflow_recommenders as tfrs

class TwoTowerModel(tf.keras.Model):
    def __init__(self, query_model, item_model):
        super().__init__()
        self.query_model = query_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            
            metrics=tfrs.metrics.FactorizedTopK(
                # Grab the batch of items and retrieve embeddings
                candidates=item_ds.batch(BATCH_SIZE).map(self.item_model)
            )
        )

    def train_step(self, batch) -> tf.Tensor:
        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.query_model(batch)
            item_embeddings = self.item_model(batch)
            loss = self.task(
                user_embeddings, 
                item_embeddings,
                compute_metrics=False, # Skip summary statistics
            )

            # Handles regularization if it has been applied 
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables) # Calculate gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables)) # update weights

        metrics = {
            "loss": loss,
            "regularization_loss": regularization_loss,
            "total_loss": total_loss
        }

        return metrics

    def test_step(self, batch) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.query_model(batch)
        item_embeddings = self.item_model(batch)

        loss = self.task(
            user_embeddings, 
            item_embeddings,
            compute_metrics=False,
        )

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

<div style="background-color: darkgreen ; color:white; padding:0px; font-size:15px">
Train

In [20]:
import tensorflow_addons as tfa

model = TwoTowerModel(query_model, item_model)

optimizer = tfa.optimizers.AdamW(0.001, learning_rate=0.01)
model.compile(optimizer=optimizer)

# tf.keras.utils.plot_model(model)

In [21]:
# Train Model
model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

2024-02-16 19:28:35.166749: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [433927]
	 [[{{node Placeholder/_5}}]]
2024-02-16 19:28:35.168066: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype string and shape [433927]
	 [[{{node Placeholder/_3}}]]


Epoch 1/5
212/212 [==============================] - ETA: 0s - loss: 15446.1301 - regularization_loss: 0.0000e+00 - total_loss: 15446.1301

2024-02-16 19:29:45.655624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [54240]
	 [[{{node Placeholder/_4}}]]


212/212 [==============================] - 73s 321ms/step - loss: 15445.6679 - regularization_loss: 0.0000e+00 - total_loss: 15445.6679 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 6608.5181 - val_regularization_loss: 0.0000e+00 - val_total_loss: 6608.5181
Epoch 2/5
212/212 [==============================] - 68s 319ms/step - loss: 14496.8359 - regularization_loss: 0.0000e+00 - total_loss: 14496.8359 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - 

<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Save Models

<div style="background-color: darkgreen ; color:white; padding:0px; font-size:15px">
Local Saves

The code below wraps the model in a module so that the raw input can be returned along with the embedding of a given input query.

In [23]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function() # Decorator allowing for input specification
    # Takes in user data and returns embedding + user data
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {"customer_id": instances["customer_id"],
                "month_sin": instances["month_sin"],
                "month_cos": instances["month_cos"],
                "query_emb": query_emb}


# Wrap only the Query Tower model into new QueryModelModule
query_model = QueryModelModule(model.query_model)

In [24]:
# Define input specifications
instances_spec={
    'customer_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='customer_id'),
    'month_sin': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_sin'),
    'month_cos': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_cos'),
    'age': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='age')
}
# Apply input specs to compute_embed()
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

# Save ModelModule w/ signature
tf.saved_model.save(query_model, "model_artifacts/query_model", signatures=signatures)

INFO:tensorflow:Assets written to: query_model/assets
2024-02-16 20:37:25,230 INFO: Assets written to: query_model/assets


In [25]:
# Save Model Locally
tf.saved_model.save(model.item_model, "model_artifacts/candidate_model")

INFO:tensorflow:Assets written to: candidate_model/assets
2024-02-16 20:38:28,511 INFO: Assets written to: candidate_model/assets


<div style="background-color: darkgreen ; color:white; padding:0px; font-size:15px">
Save To Model Registry (Hopsworks)

In [22]:
import hsml

# Connect to Hopsworks Model Registry
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [26]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Infer input schema from data.
query_model_input_schema = Schema(query_df)

# Manually specify output schema.
query_model_output_schema = Schema([{
    "name": "query_embedding",
    "type": "float32",
    "shape": [EMB_DIM]
}])

query_model_schema = ModelSchema(
    input_schema=query_model_input_schema,
    output_schema=query_model_output_schema
)

query_model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'customer_id',
    'type': 'object'},
   {'name': 'age', 'type': 'float64'},
   {'name': 'month_sin', 'type': 'float64'},
   {'name': 'month_cos', 'type': 'float64'}]},
 'output_schema': {'tensor_schema': [{'name': 'query_embedding',
    'shape': '[16]',
    'type': 'float32'}]}}

In [27]:
query_example = query_df.sample().to_dict("records")

mr_query_model = mr.tensorflow.create_model(
    name="query_model",
    description="Model that generates query embeddings from user and transaction features",
    input_example=query_example,
    model_schema=query_model_schema,
)
mr_query_model.save("query_model")

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/56 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/9927478 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/8448652 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/583 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/155 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/497 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/475285/models/query_model/1


Model(name: 'query_model', version: 1)

In [28]:
candidate_model_input_schema = Schema(item_df)

candidate_model_output_schema = Schema([{
    "name": "candidate_embedding",
    "type": "float32",
    "shape": [EMB_DIM],
}])

candidate_model_schema = ModelSchema(
    input_schema=candidate_model_input_schema,
    output_schema=candidate_model_output_schema
)

candidate_example = item_df.sample().to_dict("records")

mr_candidate_model = mr.tensorflow.create_model(
    name="candidate_model",
    description="Model that generates candidate embeddings from item features",
    input_example=candidate_example,
    model_schema=candidate_model_schema,
)
mr_candidate_model.save("candidate_model")

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/58 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/623601 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1585410 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/424 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/106 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/448 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/475285/models/candidate_model/1


Model(name: 'candidate_model', version: 1)